# Notebook activity 7.12 Using a GLM with an offset and tackling overdispersion in R

In this notebook, we'll focus on using R to fit a GLM for modelling the count response `numSeizures` from the epileptic seizures dataset. The response is defined as:

* `numSeizures`: the number of epileptic seizures experienced by the patient in the specified period when receiving the specified treatment. 

To do this, we'll make use of the variables:

* `patient`: the patient's identification number
      
* `exposure`: the exposure time (in days) which is the length of the period of observation
      
* `treatment`: the indicator of treatment given, coded 0 for the usual drug and 1 for the new drug
      
* `period`: the indicator of the period during which the patient was monitored, coded 0 for one of the two periods and 1 for the other.

The data are stored in the data frame called `seizures`.

## (a) Getting started

### (i)

Start by running the following code.

In [ ]:
# Load the M348 package
library(M348)
# Load the seizures data frame
data(seizures)
# Check the data frame has loaded OK
head(seizures)

## (b) Fitting a model with an offset

In Activity 34 (Subsection 7.3.1, Unit 7), it was suggested that a good first model to try fitting for `numSeizures` is a Poisson GLM with a log link, using the variables `treatment` and `period` as explanatory variables.

In the text just after Activity 34 it was also suggested that, since the response varies so much between patients, we should also include `patient` as a factor to account for this variation.

So, our model has the form:

`numSeizures ~ treatment + period + patient`.

However, the number of epileptic seizures counted for each patient was measured over *varying* lengths of time: these time lengths $(t_1, t_2, \ldots, t_n)$ are recorded for each observation in the variable `exposure`. So, because of these varying exposures for the response, we need to introduce an extra term into the model - the offset log(`exposure`).

It is fairly straightforward to add an offset into the model by including the `offset()` function as part of the argument of the `glm()` function. The `offset()` function takes as its argument the object to be included as an offset in the model. 

For example, the code to fit a Poisson GLM for a response variable $Y$ with an explanatory variable $x$ and an offset $\log(t)$, where $t$ is a variable containing the exposure times, is

`glm(Y ~ x + offset(log(t)), family = poisson, data = dataFrame)`.

(Assuming the vectors `Y`, `x` and `t` are in a data frame called `dataFrame`.)

### (i)

Check that `patient` is a factor.

In [ ]:
class(seizures$patient)

### (ii)

Use the empty code cell below to fit a GLM with a Poisson response and log link for the response `numSeizures` with an offset log(`exposure`) and explanatory variables `treatment`, `period` and `patient`. 

Save the resulting output in an object called `modelFit1` and display a summary of the results.

#### Solution <a class="tocSkip">

The required code and the resulting output are given below.

In [3]:
modelFit1 <- glm(numSeizures ~ offset(log(exposure)) + treatment + period + 
                 patient, family = poisson, data = seizures)
summary(modelFit1)


Call:
glm(formula = numSeizures ~ offset(log(exposure)) + treatment + 
    period + patient, family = poisson, data = seizures)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-7.5221  -0.9213   0.0019   0.8308   4.4852  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  1.08513    0.06218  17.452  < 2e-16 ***
treatment   -0.34954    0.04455  -7.845 4.32e-15 ***
period      -0.04916    0.04455  -1.103 0.269841    
patient2    -0.87043    0.11059  -7.871 3.53e-15 ***
patient3     0.09160    0.08356   1.096 0.273026    
patient4    -2.76920    0.24987 -11.083  < 2e-16 ***
patient5    -2.67238    0.23715 -11.269  < 2e-16 ***
patient6    -2.51076    0.24324 -10.322  < 2e-16 ***
patient7    -0.73367    0.10549  -6.955 3.54e-12 ***
patient8    -3.31293    0.32196 -10.290  < 2e-16 ***
patient9    -2.40514    0.20883 -11.517  < 2e-16 ***
patient10   -1.52117    0.14257 -10.669  < 2e-16 ***
patient11    1.98436    0.06408  30.966  < 2e-16 ***


### (ii)

Explain why the output for the fitted model stored in `modelFit1`, suggests that there is a problem with overdispersion here.

#### Solution <a class="tocSkip">

`modelFit1` included all of the available factors and covariates as explanatory variables, and so the residual deviance of this model, $D$, and its associated degrees of freedom, $r$, can be used to assess whether or not there seems to be a problem with overdispersion. 

From the summary output for `modelFit1`, we can see that $D = 134.97$ and $r = 13$.

Therefore,

\begin{equation*}\frac{D}{r} = \frac{134.97}{13} \simeq 10.38.\end{equation*}

Since $D/r$ is (much) greater than $2$, we can conclude that there seems to be a high degree of overdispersion.

## (c) Handling the problem of overdispersion in R

As we have just seen from the output from fitting the model `modelFit1`, the data have a serious case of overdispersion. This means that the variance observed in the response is much greater than the response variance in our model. In other words, since 

\begin{equation*}V(Y_i) = \lambda_i\end{equation*}

when $Y_i \sim \text{Poisson}(\lambda_i)$, the *observed* variance of $Y_i$ is greater than the model variance $\lambda_i$.

Note that, the fact that we had an offset term in the model has no influence on the overdispersion that is present in the data (the offset term was only used in the GLM to allow the observed values of `numSeizures` to be comparable for different `exposure` times of each `treatment`).

So, how can we tackle the overdispersion problem?

Well, following the approach of dealing with overdispersion described in Subsection 7.1 of Unit 7, we can introduce a further parameter $\phi > 0$ - the *dispersion parameter* - so that our model response variance is specified as being

\begin{equation*}V(Y_i) = \phi \lambda_i\end{equation*}

(instead of $V(Y_i) = \lambda_i$).

To do this in R, we use another distribution called `quasipoisson` (instead of `poisson`) in the `family` argument of the `glm()` function. For example, for a Poisson GLM for a response variable $Y$ with explanatory variable $x$, the code to fit a GLM with the extra dispersion parameter is:

`glm(Y ~ x, family = quasipoisson, data = dataFrame)`.

Similarly, to fit a binomial GLM with an extra dispersion parameter for a response variable $Y$ with explanatory variable $x$, we simply replace `family = binomial` in the `glm()` function with `family = quasibinomial`. That is, we use the command:

`glm(Y ~ x, family = quasibinomial)`.

### (i)

Run the code cell below to fit a Poisson GLM with a log link, including an extra dispersion parameter for the response `numSeizures` with an offset log(`exposure`) and explanatory variables `treatment`, `period` and `patient` and produce a summary of the results.

In [ ]:
modelFit2 <- glm(numSeizures ~ offset(log(exposure)) + treatment + period + 
                 patient, family = quasipoisson, data = seizures)
summary(modelFit2)

### (ii)

In the output you obtained in part (i), what value is given as the estimated value of the dispersion parameter?

#### Solution <a class="tocSkip">

From the output, the estimated value of the dispersion parameter is approximately 8.56 (to two decimal places).

## (d) How does the fitted model change when we include a dispersion parameter?

### (i)

Compare the estimates of the coefficients given in the summary outputs you obtained for `modelFit1` and `modelFit2` in parts (b)(ii) and (c)(i). What do you notice?

#### Solution <a class="tocSkip">

The estimates for the coefficients for `modelFit1` and `modelFit2` are identical. For example, for both models the estimate for `treatment` is $-0.34954$.

### (ii)

Now compare the standard errors of the coefficients in the two summary outputs. What do you notice?

#### Solution <a class="tocSkip">

The standard errors are increased when the overdispersion parameter is estimated in `modelFit2`. For example, the standard error for `treatment` is 0.04455 for `modelFit1`, but has increased to 0.13032 in `modelFit2`. 

This means that the results from the fitted model *with* the overdispersion parameter will be more realistic: without accounting for the dispersion, our confidence in our point estimates is much too great. 